# Import libraries and glance 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Data loading and initial exploration
df = pd.read_csv('/kaggle/input/restaurant-sales-data/9. Sales-Data-Analysis.csv')
print(df.head())
print(df.info())
print(df.describe())

   Order ID        Date             Product  Price  Quantity Purchase Type  \
0     10452  07-11-2022               Fries   3.49    573.07       Online    
1     10453  07-11-2022           Beverages   2.95    745.76       Online    
2     10454  07-11-2022       Sides & Other   4.99    200.40     In-store    
3     10455  08-11-2022             Burgers  12.99    569.67     In-store    
4     10456  08-11-2022  Chicken Sandwiches   9.95    201.01     In-store    

  Payment Method             Manager    City  
0      Gift Card    Tom      Jackson  London  
1      Gift Card         Pablo Perez  Madrid  
2      Gift Card       Joao    Silva  Lisbon  
3    Credit Card       Walter Muller  Berlin  
4    Credit Card       Walter Muller  Berlin  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        254 non-null    int64  
 1   Date 

In [2]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
print("=== DATA OVERVIEW ===")
print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
print("=== MISSING VALUES ===")
print(df.isnull().sum())
print("=== UNIQUE VALUES PER COLUMN ===")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

=== DATA OVERVIEW ===
Dataset shape: (254, 9)
Memory usage: 104.70 KB
=== MISSING VALUES ===
Order ID          0
Date              0
Product           0
Price             0
Quantity          0
Purchase Type     0
Payment Method    0
Manager           0
City              0
dtype: int64
=== UNIQUE VALUES PER COLUMN ===
Order ID: 254 unique values
Date: 53 unique values
Product: 5 unique values
Price: 7 unique values
Quantity: 29 unique values
Purchase Type: 3 unique values
Payment Method: 3 unique values
Manager: 14 unique values
City: 5 unique values


* No missing value at all.Good for you XD

# Data Type Corrections and Feature Engineering

In [3]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Create additional date features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['WeekOfYear'] = df['Date'].dt.isocalendar().week

# Calculate total revenue per order
df['Total_Revenue'] = df['Price'] * df['Quantity']

print("=== DATE RANGE ===")
print(f"From: {df['Date'].min()} to {df['Date'].max()}")
print(f"Total days: {(df['Date'].max() - df['Date'].min()).days + 1}")


=== DATE RANGE ===
From: 2022-11-07 00:00:00 to 2022-12-29 00:00:00
Total days: 53
